In [ ]:
#Objective: predicting the trip duration accurately

Extract data we've been given and divide into reasonable chunks

In [5]:
import pandas as pd

chunk_size = 50000 
file_path = 'training_dataset/training_dataset.csv'

chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunks.append(chunk)

data_frame = pd.concat(chunks, ignore_index=True)


Process the data. Remove whatever that's uncessary and check for missing values. Either remove the missing or replace them. 

But first, check what we're missing and working with.

In [12]:
# Check for missing values
missing_values = data_frame.isnull().sum()
print("Number of missing values are: ", missing_values) 

print("-------------------------------------------------")

# Show summary statistics
print(data_frame.info())

Number of missing values are:  ID                       0
vendorid                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
trip_distance            0
pulocationid             0
dolocationid             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
airport_fee              0
duration                 0
dtype: int64
-------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32701098 entries, 0 to 32701097
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   vendorid               int64  
 2   tpep_pickup_datetime   object 
 3   tpep_dropoff_datetime  object 
 4   trip_distance          float64
 5   pulocationid           int64  
 6   dolocationid           int64  
 7   pay

We're missing a lot of data from these columns: 

1. passenger_count          3155336

2. ratecodeid               3155336

3. store_and_fwd_flag       3155336

4. congestion_surcharge     3155336

5. airport_fee              3155336


The first four columns are useless for the objective at hand. We'll drop them. 

Whilst we're at it we will remove all other columns that are not useful.

In [14]:
# YOUR CODE HERE
data_frame.drop(columns=["ID", "airport_fee", "vendorid", "extra", "mta_tax", "tip_amount", "tolls_amount", 
                         "improvement_surcharge", "fare_amount"], inplace=True)
data_frame.fillna(0, inplace=True)


Missing values have been addressed as well as useless columns. Now, we'll process the data that is relevant. 

In [15]:
print(data_frame.describe())

       trip_distance  pulocationid  dolocationid  payment_type  total_amount  \
count   3.270110e+07  3.270110e+07  3.270110e+07  3.270110e+07  3.270110e+07   
mean    4.908534e+00  1.642556e+02  1.634615e+02  1.110001e+00  2.785063e+01   
std     4.088654e+02  6.433638e+01  6.960137e+01  6.450934e-01  8.670093e+01   
min     0.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00 -2.265450e+03   
25%     1.020000e+00  1.320000e+02  1.130000e+02  1.000000e+00  1.575000e+01   
50%     1.760000e+00  1.610000e+02  1.620000e+02  1.000000e+00  2.100000e+01   
75%     3.360000e+00  2.330000e+02  2.340000e+02  1.000000e+00  3.050000e+01   
max     3.986086e+05  2.650000e+02  2.650000e+02  5.000000e+00  3.355509e+05   

           duration  
count  3.270110e+07  
mean   1.045067e+03  
std    2.081190e+03  
min    0.000000e+00  
25%    4.690000e+02  
50%    7.770000e+02  
75%    1.261000e+03  
max    5.854240e+05  
